In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import pickle
from catboost import CatBoostClassifier
warnings.filterwarnings("ignore")

In [7]:
from sklearn.model_selection import train_test_split
import pickle
from sklearn import tree
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report,f1_score

class AdaBoostClassifier:
    '''A simple AdaBoost Classifier.'''

    def __init__(self, n_weakers_limit):
        self.weights = []
        self.classifiers = []
        self.n_weakers_limit = n_weakers_limit
        self.ws = []
        pass

    def is_good_enough(self,X_train,y_train):
        #if((len(self.classifiers) > self.n_weakers_limit) or
        #        accuracy_score(y_train, self.predict(X_train))>=0.9):
        if(len(self.classifiers) > self.n_weakers_limit):
            C_report = classification_report(y_train, self.predict(X_train), digits=4, output_dict=True)
            print(classification_report(y_train, self.predict(X_train), digits=4))
            report = f1_score(y_train, self.predict(X_train), average=None)
            print('Score: ', report)
            df = pd.DataFrame(C_report).transpose()
            df.to_csv("classifier_report.csv", index= True)
            return True
        else:
            # print("准确率：" + str(accuracy_score(y_train, self.predict(X_train))))
            print(classification_report(y_train, self.predict(X_train), digits=4))
            report = f1_score(y_train, self.predict(X_train), average=None)
            print('Score: ', report)
            return False
        pass

    def fit(self,X_train,y_train):
        w = np.ones((1, len(X_train))) / len(y_train)
        flag = 0
        while flag == 0 or (not (self.is_good_enough(X_train, y_train))):
            # for i in range(2):
            # 定义并训练基分类器决策树
            flag = 1
            clf = tree.DecisionTreeClassifier \
                (class_weight="balanced", criterion='gini', max_depth=1, splitter="random", random_state=0)
            predit_tree = clf.fit(X_train, y_train, w.reshape(-1))
            # predit_tree = clf.fit(X_train, y_train)
            self.classifiers.append(predit_tree)
            # 计算错误率
            correct_rate = predit_tree.score(X_train, y_train)
            error_rate = 1 - correct_rate
            # 计算分类器权重
            alpha = 1 / 2 * np.log((1 - error_rate) / error_rate)
            # 存储分类器权重
            self.weights.append(alpha)
            # 计算预测结果
            result = predit_tree.predict(X_train).reshape(-1, 1)
            # 更新数据集权重参数
            first_part = -alpha * y_train
            second_part = np.exp(first_part * result)
            third_part = (w.reshape(-1, 1) * second_part)
            z_t = np.sum(third_part)
            w = w.reshape(-1, 1) * second_part / z_t
            self.ws.append(w)
            print("alpha=", alpha)
        pass


    def predict_scores(self, X_train):
        '''Calculate the weighted sum score of the whole base classifiers for given samples.

        Args:
            X: An ndarray indicating the samples to be predicted, which shape should be (n_samples,n_features).

        Returns:
            An one-dimension ndarray indicating the scores of differnt samples, which shape should be (n_samples,1).
        '''
        scores = np.zeros((1, len(X_train)))
        for i in range(len(self.classifiers)):
            scores +=  self.classifiers[i].predict(X_train)*self.weights[i]    
            
        return scores
        pass

    def predict(self, X_train, threshold=0):
        '''Predict the catagories for geven samples.

        Args:
            X: An ndarray indicating the samples to be predicted, which shape should be (n_samples,n_features).
            threshold: The demarcation number of deviding the samples into two parts.

        Returns:
            An ndarray consists of predicted labels, which shape should be (n_samples,1).
        '''
        scores = self.predict_scores(X_train)
        y_train = np.where(scores>threshold, 1, -1)
        return y_train.reshape(-1,1)
        pass

    @staticmethod
    def save(model, filename):
        with open(filename, "wb") as f:
            pickle.dump(model, f)

    @staticmethod
    def load(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

In [2]:
train = pd.read_csv('is_train_data/is_train_20190701.txt')
train = pd.merge(train[['link','current_slice_id','future_slice_id']][(train.label == 2)],train,how='left') 


In [3]:
for i in range(2, 10):
    train1 = pd.read_csv('is_train_data/is_train_2019070'+str(i)+'.txt')
    train = train.append(pd.merge(train1[['link','current_slice_id','future_slice_id']][(train1.label == 2)],train1,how='left'))


In [4]:
train

,link,current_slice_id,future_slice_id,label,time_diff,now_speed,now_eta,now_cnt,now_state,current_speed_min,...,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7_state_zhong,his_7_state_max,his_7_state_min,is_high
0,33503,409,437,2,28,36.5,34.1,5,1,35.187500,...,16.656250,3.148438,3,8,4.6,1.743560,2,2,2,0
1,277708,536,547,2,11,24.3,7.4,20,2,3.599609,...,8.296875,0.877930,27,32,29.0,1.788854,3,4,3,1
2,34756,539,560,2,21,6.0,12.9,19,3,5.000000,...,13.359375,1.260742,20,28,24.4,3.136877,3,4,3,1
3,273215,200,221,2,21,55.2,36.7,7,1,52.312500,...,14.054688,1.059570,15,21,18.4,2.154066,2,3,1,1
4,639183,241,247,2,6,6.5,12.5,42,3,5.898438,...,16.625000,2.701172,25,40,32.4,5.535341,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22189,171823,337,352,2,15,16.6,14.8,9,2,16.000000,...,9.796875,0.363525,6,8,7.4,0.800000,2,2,2,0
22190,209839,589,612,2,23,3.1,1.9,6,4,3.099609,...,24.421875,0.830078,7,16,11.4,2.939388,1,1,1,0
22191,370809,604,607,2,3,2.9,3.0,7,4,2.699219,...,22.015625,1.762695,11,13,12.2,0.748331,1,2,1,0
22192,162241,678,680,2,2,28.3,26.8,9,1,28.296875,...,26.031250,4.699219,1,4,2.4,1.356466,1,1,1,0


In [12]:
y = np.where(y == 2 , 1, -1)

In [13]:
AdaBoost = AdaBoostClassifier(20)

In [19]:
from sklearn.model_selection import train_test_split
# X_train, X_validation, y_train, y_validation = train_test_split(train.loc[:,'time_diff':'width'],y,test_size=0.25 , random_state=1234)
X_train, X_validation, y_train, y_validation = train_test_split(train,y,test_size=0.2 , random_state=1234)

In [20]:
AdaBoost.fit(X_train, y_train.reshape(-1,1))

alpha= 0.5836402861116483
              precision    recall  f1-score   support

          -1     0.9885    0.7627    0.8611    390270
           1     0.1064    0.7606    0.1866     14492

    accuracy                         0.7627    404762
   macro avg     0.5474    0.7617    0.5239    404762
weighted avg     0.9569    0.7627    0.8369    404762

Score:  [0.8610533  0.18664861]
alpha= 0.9848892640318742
              precision    recall  f1-score   support

          -1     0.9885    0.7627    0.8611    390270
           1     0.1064    0.7606    0.1866     14492

    accuracy                         0.7627    404762
   macro avg     0.5474    0.7617    0.5239    404762
weighted avg     0.9569    0.7627    0.8369    404762

Score:  [0.8610533  0.18664861]
alpha= -0.9848892640318742
              precision    recall  f1-score   support

          -1     0.9885    0.7627    0.8611    390270
           1     0.1064    0.7606    0.1866     14492

    accuracy                         0.